In [1]:
import pandas as pd
from spectral_cube import SpectralCube
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
fits_path = ".data/spw17.CI.clean.freq.fits"

In [3]:
def fits_to_dataframe_sequential(fits_file):
    cube = SpectralCube.read(fits_file)
    
    df_list = []

    # Iterate over the spectral axis (velocity axis)
    for i in range(cube.shape[0]):
        # Slice one spectral frame at a time
        slab = cube[i, :, :]  # shape: (y, x)

        # Get world coordinates for this frame
        world = slab.wcs.pixel_to_world_values(
            *np.meshgrid(np.arange(cube.shape[2]),  # x (RA)
                         np.arange(cube.shape[1]),  # y (Dec)
                         indexing='xy')
        )

        ra = world[0].flatten()
        dec = world[1].flatten()
        velo = cube.spectral_axis[i].value  # Single velocity value for this slice
        intensity = slab.filled_data[:].value.flatten()

        # Build DataFrame for this slab
        df_slice = pd.DataFrame({
            'Velocity': velo,
            'RA': ra,
            'Dec': dec,
            'Intensity': intensity
        })

        df_list.append(df_slice)

    return pd.concat(df_list, ignore_index=True)


In [4]:
df = fits_to_dataframe_sequential(fits_path)
df = df[df["Intensity"]>0]

In [5]:
df

,Velocity,RA,Dec,Intensity
7200008,1.434146e+11,132.394927,2.239444,1.507095e-07
7200009,1.434146e+11,132.394917,2.239444,1.875926e-07
7200019,1.434146e+11,132.394820,2.239444,4.076169e-05
7200020,1.434146e+11,132.394810,2.239444,6.182483e-05
7200021,1.434146e+11,132.394800,2.239444,6.052495e-05
...,...,...,...,...
178559987,1.452584e+11,132.383455,2.251101,1.614566e-04
178559988,1.452584e+11,132.383446,2.251101,1.474867e-04
178559989,1.452584e+11,132.383436,2.251101,1.095917e-04
178559990,1.452584e+11,132.383426,2.251101,6.442113e-05


In [8]:
def visualize_with_open3d_filtered(df, intensity_min=None, intensity_max=None, velocity_scale=1):
    # Filter the DataFrame based on intensity
    if intensity_min is not None:
        df = df[df['Intensity'] >= intensity_min].copy()  # Create a copy here
    if intensity_max is not None:
        df = df[df['Intensity'] <= intensity_max].copy()  # Create a copy here

    # Apply standardized scaling to the velocity
    df['std_Velocity'] = (df['Velocity'] - np.mean(df['Velocity'])) / np.std(df['Velocity'])
    
    # Logarithmic Velocity
    df["log_velocity"] = np.log(df['Velocity'] - df['Velocity'].min() + 1e-9)

    # Prepare points for visualization (RA, Dec, log-scaled velocity)
    points = np.vstack((-df['RA'], df['Dec'], df['log_velocity'] * velocity_scale)).T

    # Create an Open3D point cloud object
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points)

    # Normalize intensity for color mapping
    intensity = (df['Intensity'] - df['Intensity'].mean()) / df['Intensity'].std()
    colors = plt.cm.plasma(intensity)[:, :3]

    # Apply colors to the point cloud
    point_cloud.colors = o3d.utility.Vector3dVector(colors)

    # Visualize the point cloud
    o3d.visualization.draw_geometries([point_cloud])

# Example usage
# Assume df is your Pandas DataFrame obtained from the FITS file
# df = fits_to_dataframe('your_file.fits')
visualize_with_open3d_filtered(df, intensity_min=df['Intensity'].quantile(0.995), intensity_max=None, velocity_scale=0.001)

MESA: error: ZINK: failed to choose pdev
glx: failed to create drisw screen
